In [1]:
# Gerekli kütüphaneleri içe aktar
import os
from pathlib import Path
import requests
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

# Ortak klasörleri tanımla
RAW_DIR = Path("../data/raw")
ELEV_DIR = RAW_DIR / "elevation"
LIGHT_DIR = RAW_DIR / "lightpollution"
CLIMATE_DIR = RAW_DIR / "climate"
BOUNDARY_DIR = RAW_DIR / "external" / "boundaries"

# Klasörleri oluştur
for folder in [ELEV_DIR, LIGHT_DIR, CLIMATE_DIR, BOUNDARY_DIR]:
    folder.mkdir(parents=True, exist_ok=True)

print("Klasör yapısı hazır.")


Klasör yapısı hazır.


In [2]:
# Zaten indirilmiş SRTM .tif dosyalarını listele
tif_files = list(ELEV_DIR.glob("*.tif"))
print(f"{len(tif_files)} SRTM dosyası bulundu.")


172 SRTM dosyası bulundu.


In [3]:
# VIIRS ışık kirliliği verisini kontrol et
viirs_file = LIGHT_DIR / "VNL_npp_2024_global_vcmslcfg_v2_c202502261200.median_masked.dat.tif"
if viirs_file.exists():
    print("VIIRS ışık kirliliği verisi mevcut.")
else:
    print("⚠️ VIIRS ışık kirliliği verisi bulunamadı. Elle indirmeniz gerekebilir.")


VIIRS ışık kirliliği verisi mevcut.


In [2]:
from pathlib import Path

monthly_climate_dir = Path("../data/raw/climate/monthly")

# Klasör var mı kontrol et, yoksa oluştur
if not monthly_climate_dir.exists():
    monthly_climate_dir.mkdir(parents=True)
    print(f"✅ Klasör oluşturuldu: {monthly_climate_dir}")
else:
    print(f"✅ Klasör zaten mevcut: {monthly_climate_dir}")



✅ Klasör oluşturuldu: ..\data\raw\climate\monthly


In [15]:
import geopandas as gpd
from shapely.geometry import mapping

BOUNDARY_PATH = "../data/external/boundaries/turkey_boundary.geojson"

# Türkiye sınırlarını yükle
gdf_turkey = gpd.read_file(BOUNDARY_PATH)
gdf_turkey = gdf_turkey.to_crs("EPSG:4326")

# Rasterio ile uyumlu hale getir
turkey_geom = [mapping(geom) for geom in gdf_turkey.geometry]

print("✅ Türkiye sınırları yüklendi ve EPSG:4326'e dönüştürüldü.")


✅ Türkiye sınırları yüklendi ve EPSG:4326'e dönüştürüldü.


In [17]:
import requests
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
from time import sleep

# 📁 1. Klasörleri oluştur
climate_dir = Path("../data/raw/climate/monthly")
climate_dir.mkdir(parents=True, exist_ok=True)

# 📍 2. Türkiye sınırlarını yükle (GeoJSON kullanalım)
turkey_gdf = gpd.read_file("../data/external/boundaries/turkey_boundary.geojson")

# 🌐 3. Grid oluştur (0.5° çözünürlükte)
lat_range = np.arange(36.0, 42.01, 0.5)
lon_range = np.arange(26.0, 45.01, 0.5)
grid_points = [Point(lon, lat) for lat in lat_range for lon in lon_range]
grid_gdf = gpd.GeoDataFrame(geometry=grid_points, crs="EPSG:4326")

# 🗺️ 4. Türkiye içindeki noktaları filtrele
turkey_union = turkey_gdf.unary_union
grid_within = grid_gdf[grid_gdf.within(turkey_union)]
print(f"📌 Türkiye içinde {len(grid_within)} nokta bulundu.")

# 🌤️ 5. API parametreleri
base_url = "https://power.larc.nasa.gov/api/temporal/climatology/point"
parameters = "CLOUD_AMT,RH2M,T2M,WS2M,ALLSKY_SFC_SW_DWN,PAR,TO3,PS"

# 📃 6. Veri olmayan noktalar log dosyası
no_data_log = Path("veri_yok_noktalar.txt")
no_data_log.write_text("")  # varsa temizle

# 🚀 7. Veri indirme döngüsü
for i, point in enumerate(grid_within.geometry):
    lat = round(point.y, 2)
    lon = round(point.x, 2)
    filename = climate_dir / f"climate_{lat:.2f}_{lon:.2f}.csv"
    if filename.exists():
        continue

    url = (
        f"{base_url}?parameters={parameters}"
        f"&community=RE&longitude={lon:.2f}&latitude={lat:.2f}&format=CSV"
    )

    try:
        response = requests.get(url, timeout=15)
        if response.status_code == 200 and "BEGIN HEADER" in response.text:
            with open(filename, "w", encoding="utf-8") as f:
                f.write(response.text)
        else:
            with open(no_data_log, "a", encoding="utf-8") as f:
                f.write(f"{lat},{lon} - NO DATA\n")

    except requests.exceptions.Timeout:
        with open(no_data_log, "a", encoding="utf-8") as f:
            f.write(f"{lat},{lon} - TIMEOUT\n")

    except Exception as e:
        with open(no_data_log, "a", encoding="utf-8") as f:
            f.write(f"{lat},{lon} - ERROR: {e}\n")

    if i % 20 == 0:
        print(f"🔄 {i}/{len(grid_within)} noktadan veri alındı...")

    sleep(0.5)  # API'yi yormamak için

print("✅ Tüm işlemler tamamlandı.")


📌 Türkiye içinde 321 nokta bulundu.
🔄 0/321 noktadan veri alındı...
🔄 20/321 noktadan veri alındı...
🔄 40/321 noktadan veri alındı...
🔄 60/321 noktadan veri alındı...
🔄 80/321 noktadan veri alındı...
🔄 100/321 noktadan veri alındı...
🔄 120/321 noktadan veri alındı...
🔄 140/321 noktadan veri alındı...
🔄 160/321 noktadan veri alındı...
🔄 180/321 noktadan veri alındı...
🔄 200/321 noktadan veri alındı...
🔄 220/321 noktadan veri alındı...
🔄 240/321 noktadan veri alındı...
🔄 260/321 noktadan veri alındı...
🔄 280/321 noktadan veri alındı...
🔄 300/321 noktadan veri alındı...
🔄 320/321 noktadan veri alındı...
✅ Tüm işlemler tamamlandı.


In [12]:
import requests
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
from time import sleep

# 📁 1. Klasörleri hazırla
climate_dir = Path("../data/raw/climate/monthly")
climate_dir.mkdir(parents=True, exist_ok=True)

# 📍 2. Türkiye sınırlarını yükle
turkey_gdf = gpd.read_file("../data/external/boundaries/turkey_boundary.geojson")

# 🌐 3. Grid oluştur (0.5° çözünürlükte)
lat_range = np.arange(36.0, 42.01, 0.5)
lon_range = np.arange(26.0, 45.01, 0.5)
grid_points = [Point(lon, lat) for lat in lat_range for lon in lon_range]
grid_gdf = gpd.GeoDataFrame(geometry=grid_points, crs="EPSG:4326")

# 🗺️ 4. Türkiye içindeki noktaları al
turkey_union = turkey_gdf.unary_union
grid_within = grid_gdf[grid_gdf.within(turkey_union)]
print(f"📌 Türkiye içinde {len(grid_within)} nokta bulundu.")

# 🌤️ 5. Parametre grupları
param_groups = {
    "grp1": ["CLOUD_AMT", "CLOUD_AMT_DAY", "CLOUD_AMT_NIGHT", "CLRSKY_DAYS", "ALLSKY_SFC_SW_DWN", "CLRSKY_SFC_SW_DWN", "ALLSKY_SFC_PAR_TOT"],
    "grp2": ["T2M", "T2M_MAX", "T2M_MIN", "T2M_RANGE", "T2MWET", "T2MDEW", "RH2M", "QV2M", "TQV"],
    "grp3": ["WS2M", "WS2M_MAX", "WD2M", "PS", "SLP"],
    "grp4": ["PRECTOT", "TO3", "TS", "UVDEF"]
}

base_url = "https://power.larc.nasa.gov/api/temporal/climatology/point"
log_file = Path("veri_yok_noktalar.txt")
log_file.write_text("")

# 🚀 6. İndirme döngüsü
for i, point in enumerate(grid_within.geometry):
    lat = round(point.y, 2)
    lon = round(point.x, 2)

    for group_name, params in param_groups.items():
        param_str = ",".join(params)
        filename = climate_dir / f"climate_{lat:.2f}_{lon:.2f}_{group_name}.csv"
        if filename.exists():
            continue

        url = (
            f"{base_url}?parameters={param_str}"
            f"&community=RE&longitude={lon:.2f}&latitude={lat:.2f}&format=CSV"
        )

        try:
            response = requests.get(url, timeout=15)
            if response.status_code == 200 and "BEGIN HEADER" in response.text:
                with open(filename, "w", encoding="utf-8") as f:
                    f.write(response.text)
            else:
                with open(log_file, "a", encoding="utf-8") as f:
                    f.write(f"{lat},{lon},{group_name} - NO DATA\n")

        except requests.exceptions.Timeout:
            with open(log_file, "a", encoding="utf-8") as f:
                f.write(f"{lat},{lon},{group_name} - TIMEOUT\n")

        except Exception as e:
            with open(log_file, "a", encoding="utf-8") as f:
                f.write(f"{lat},{lon},{group_name} - ERROR: {e}\n")

        sleep(0.3)  # Çok yüklenmemek için biraz bekle

    if i % 20 == 0:
        print(f"🔄 {i}/{len(grid_within)} noktadan veri indirildi...")

print("✅ Tüm işlemler tamamlandı.")


C:\Users\yalaz\AppData\Local\Temp\ipykernel_18692\4233786690.py:22: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  turkey_union = turkey_gdf.unary_union


📌 Türkiye içinde 321 nokta bulundu.
🔄 0/321 noktadan veri indirildi...
🔄 20/321 noktadan veri indirildi...
🔄 40/321 noktadan veri indirildi...
🔄 60/321 noktadan veri indirildi...
🔄 80/321 noktadan veri indirildi...
🔄 100/321 noktadan veri indirildi...
🔄 120/321 noktadan veri indirildi...
🔄 140/321 noktadan veri indirildi...
🔄 160/321 noktadan veri indirildi...
🔄 180/321 noktadan veri indirildi...
🔄 200/321 noktadan veri indirildi...
🔄 220/321 noktadan veri indirildi...
🔄 240/321 noktadan veri indirildi...
🔄 260/321 noktadan veri indirildi...
🔄 280/321 noktadan veri indirildi...
🔄 300/321 noktadan veri indirildi...
🔄 320/321 noktadan veri indirildi...
✅ Tüm işlemler tamamlandı.


In [15]:
import requests
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
from time import sleep

# 📁 1. Klasör ve yol tanımları
climate_dir = Path("../data/raw/climate/monthly")
climate_dir.mkdir(parents=True, exist_ok=True)

# 📍 2. Türkiye sınırları (GeoJSON)
turkey_gdf = gpd.read_file("../data/external/boundaries/turkey_boundary.geojson")
turkey_union = turkey_gdf.geometry.unary_union  # Uyarıya rağmen işlevsel

# 🌐 3. Grid (0.5°)
lat_range = np.arange(36.0, 42.01, 0.5)
lon_range = np.arange(26.0, 45.01, 0.5)
grid_points = [Point(lon, lat) for lat in lat_range for lon in lon_range]
grid_gdf = gpd.GeoDataFrame(geometry=grid_points, crs="EPSG:4326")

# 📌 4. Türkiye içindeki noktalar
grid_within = grid_gdf[grid_gdf.within(turkey_union)]
print(f"📌 Türkiye içinde {len(grid_within)} nokta bulundu.")

# 🌧️ 5. grp4 parametreleri
grp4_params = ["PRECTOT", "TO3", "TS"]
base_url = "https://power.larc.nasa.gov/api/temporal/climatology/point"

# 📃 6. Log dosyası
log_file = Path("veri_yok_grp4.txt")
log_file.write_text("")  # varsa temizle

# 🚀 7. İndirme döngüsü
for i, point in enumerate(grid_within.geometry):
    lat = round(point.y, 2)
    lon = round(point.x, 2)
    filename = climate_dir / f"climate_{lat:.2f}_{lon:.2f}_grp4.csv"
    if filename.exists():
        continue

    params_str = ",".join(grp4_params)
    url = (
        f"{base_url}?parameters={params_str}"
        f"&community=RE&longitude={lon:.2f}&latitude={lat:.2f}&format=CSV"
    )

    try:
        response = requests.get(url, timeout=15)
        if response.status_code == 200 and "BEGIN HEADER" in response.text:
            with open(filename, "w", encoding="utf-8") as f:
                f.write(response.text)
        else:
            with open(log_file, "a", encoding="utf-8") as f:
                f.write(f"{lat},{lon} - NO DATA\n")

    except requests.exceptions.Timeout:
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(f"{lat},{lon} - TIMEOUT\n")

    except Exception as e:
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(f"{lat},{lon} - ERROR: {e}\n")

    if i % 20 == 0:
        print(f"🔄 {i}/{len(grid_within)} noktadan veri indirildi...")

    sleep(0.5)

print("✅ grp4 verileri indirildi.")


C:\Users\yalaz\AppData\Local\Temp\ipykernel_18692\2085729030.py:14: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  turkey_union = turkey_gdf.geometry.unary_union  # Uyarıya rağmen işlevsel


📌 Türkiye içinde 321 nokta bulundu.
🔄 0/321 noktadan veri indirildi...
🔄 20/321 noktadan veri indirildi...
🔄 40/321 noktadan veri indirildi...
🔄 60/321 noktadan veri indirildi...
🔄 80/321 noktadan veri indirildi...
🔄 100/321 noktadan veri indirildi...
🔄 120/321 noktadan veri indirildi...
🔄 140/321 noktadan veri indirildi...
🔄 160/321 noktadan veri indirildi...
🔄 180/321 noktadan veri indirildi...
🔄 200/321 noktadan veri indirildi...
🔄 220/321 noktadan veri indirildi...
🔄 240/321 noktadan veri indirildi...
🔄 260/321 noktadan veri indirildi...
🔄 280/321 noktadan veri indirildi...
🔄 300/321 noktadan veri indirildi...
🔄 320/321 noktadan veri indirildi...
✅ grp4 verileri indirildi.


In [20]:
from pathlib import Path

# 📁 İklim veri klasörü
climate_dir = Path("../data/raw/climate/monthly")

# 📄 Dosya adlarını kontrol etmek için tüm CSV dosyalarını al
all_files = list(climate_dir.glob("*.csv"))
all_filenames = [f.name for f in all_files]

# 🔍 Noktaları ve grupları ayıkla
from collections import defaultdict

# Dosya adı formatı: climate_<lat>_<lon>_grpX.csv
file_map = defaultdict(set)

for fname in all_filenames:
    if "grp" in fname:
        parts = fname.replace(".csv", "").split("_")
        latlon = f"{parts[1]}_{parts[2]}"
        group = parts[3]
        file_map[latlon].add(group)

# 📊 Kontrol
complete_points = []
incomplete_points = []

for latlon, groups in file_map.items():
    if {"grp1", "grp2", "grp3", "grp4"}.issubset(groups):
        complete_points.append(latlon)
    else:
        incomplete_points.append((latlon, sorted(groups)))

print(f"✅ Tüm dosyaları olan noktalar: {len(complete_points)}")
print(f"⚠️ Eksik dosyalı noktalar: {len(incomplete_points)}")

# Detaylı rapor
if incomplete_points:
    print("\nEksik grupları olan noktalar örnekleri (ilk 10):")
    for pt, available in incomplete_points[:10]:
        missing = {"grp1", "grp2", "grp3", "grp4"} - set(available)
        print(f"📍 Nokta: {pt} | Eksik: {', '.join(sorted(missing))}")


✅ Tüm dosyaları olan noktalar: 321
⚠️ Eksik dosyalı noktalar: 0


In [27]:
import pandas as pd
from pathlib import Path

# ➕ Yardımcı fonksiyon: ondalıklı range
def frange(start, stop, step):
    while start <= stop:
        yield round(start, 2)
        start += step

# 📂 Klasör yolları
monthly_dir = Path("../data/raw/climate/monthly")
merged_dir = Path("../data/processed/climate")
merged_dir.mkdir(parents=True, exist_ok=True)

# 🧱 Gruplar ve grid noktaları
groups = ["grp1", "grp2", "grp3", "grp4"]
lat_range = list(frange(36.0, 42.0, 0.5))
lon_range = list(frange(26.0, 45.0, 0.5))
grid_points = [(lat, lon) for lat in lat_range for lon in lon_range]

merged_data = []

# 🔁 Noktalar üzerinde döngü
for lat, lon in grid_points:
    all_parts = []
    missing = False

    for grp in groups:
        file_path = monthly_dir / f"climate_{lat:.2f}_{lon:.2f}_{grp}.csv"
        if not file_path.exists():
            missing = True
            break

        try:
            # HEADER'dan sonrası
            with open(file_path, encoding="utf-8") as f:
                lines = f.readlines()

            for i, line in enumerate(lines):
                if "-END HEADER-" in line:
                    data_start = i + 1
                    break
            else:
                print(f"❌ HEADER bulunamadı: {file_path.name}")
                missing = True
                break

            df = pd.read_csv(file_path, skiprows=data_start)
            df = df.set_index("PARAMETER").T
            df.columns = [f"{col}_{grp}" for col in df.columns]
            df.reset_index(drop=True, inplace=True)
            all_parts.append(df)

        except Exception as e:
            print(f"❌ Hata ({file_path.name}): {e}")
            missing = True
            break

    if not missing:
        combined = pd.concat(all_parts, axis=1)
        combined.insert(0, "latitude", lat)
        combined.insert(1, "longitude", lon)
        merged_data.append(combined)

# 📦 Tüm verileri birleştir
final_df = pd.concat(merged_data, ignore_index=True)
output_path = merged_dir / "merged_climate_data.csv"
final_df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"✅ Tüm veriler birleştirildi ve kaydedildi: {output_path}")


✅ Tüm veriler birleştirildi ve kaydedildi: ..\data\processed\climate\merged_climate_data.csv


In [41]:
import os

def print_directory_tree(start_path, indent=""):
    """
    Verilen klasör yolundaki dosya ve klasör yapısını ağaç şeklinde yazdırır.
    """
    try:
        for item in sorted(os.listdir(start_path)):
            path = os.path.join(start_path, item)
            if os.path.isdir(path):
                print(f"{indent}📁 {item}/")
                print_directory_tree(path, indent + "    ")
            else:
                print(f"{indent}📄 {item}")
    except FileNotFoundError:
        print(f"❌ Klasör bulunamadı: {start_path}")
    except PermissionError:
        print(f"⚠️ Erişim engellendi: {start_path}")

# Kök klasör yolunu ayarla
project_root = r"C:\Users\yalaz\gozlemevi_yer_secimi"


# Çalıştır
print(f"\n📦 Proje Yapısı: {os.path.abspath(project_root)}\n")
print_directory_tree(project_root)



📦 Proje Yapısı: C:\Users\yalaz\gozlemevi_yer_secimi

📄 .gitignore
📁 .ipynb_checkpoints/
📄 README.md
📁 config/
📁 data/
    📁 external/
        📁 .ipynb_checkpoints/
        📁 boundaries/
            📁 .ipynb_checkpoints/
                📄 turkey_boundary-checkpoint.geojson
            📄 turkey_boundary.geojson
    📁 processed/
        📁 .ipynb_checkpoints/
        📁 climate/
            📁 .ipynb_checkpoints/
                📄 merged_climate_data-checkpoint.csv
            📄 merged_climate_data.csv
        📄 srtm_turkiye_cropped.tif
        📄 srtm_turkiye_mosaic.tif
        📄 viirs_light_2024_turkey.tif
    📁 raw/
        📁 .ipynb_checkpoints/
        📁 climate/
            📁 .ipynb_checkpoints/
            📁 monthly/
                📁 .ipynb_checkpoints/
                    📄 climate_36.00_36.00_grp1-checkpoint.csv
                    📄 climate_36.00_36.00_grp2-checkpoint.csv
                    📄 climate_36.00_36.00_grp3-checkpoint.csv
                    📄 climate_36.50_29.50_grp1-ch